In [13]:
import pickle
import lightgbm

import pandas as pd

from sklearn.model_selection import TimeSeriesSplit, KFold
from catboost import CatBoostRegressor, Pool

from lightgbm import LGBMRegressor, Dataset
from lightgbm import train as Train
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

from xgboost import XGBRegressor, DMatrix
from xgboost import train as xgb_train

# Подгрузка моделей

In [14]:
import pickle
# считаем лучшие модели, в нашем случае это xgboost по фолдам
lgbm_by_fold = []
catboost_by_fold = []
xgb_by_fold = []
for i in range(5):
    with open(f'models/xgb_model_{i}.pkl', 'rb') as f:
        xgb_by_fold.append(pickle.load(f))

    lgbm_by_fold.append(lightgbm.Booster(model_file='models/best_lgbm.txt'))

    best_catboost = CatBoostRegressor()
    catboost_by_fold.append(best_catboost.load_model('models/best_catboost'))


In [22]:
test = pd.read_csv('data/mars-public_test-regr.csv')
private_test = pd.read_csv('mars_final_private/mars-private_test-reg.csv')

In [23]:
columns = ['num_test', 'signal_module', 'measure_type', 'num_impulse', 'hor',
           'ver', 'noise', 'azimuth', 'u.m.', 'sec', 'range']
new_test = test.copy()
new_test.columns = columns
new_test = new_test.drop(['hor', 'ver', 'measure_type', 'num_impulse'], axis = 1)

предикт на паблике

In [ ]:
preds = []
test_dmatrix = DMatrix(new_test.drop('num_test', axis = 1))
for xgb_model in xgb_by_fold:
    preds.append(xgb_model.predict(test_dmatrix))
xgb_pred_by_fold = np.mean(preds, axis = 0)

preds = []
for lgbm_i in lgbm_by_fold:
    preds.append(lgbm_i.predict(new_test))
lgbm_pred_by_fold = np.mean(preds, axis = 0)

preds = []

for catboost_i in catboost_by_fold:
    preds.append(catboost_i.predict(new_test))
catboost_pred_by_fold = np.mean(preds, axis = 0)

In [ ]:
koef = 1/(46.75517 + 47.82269 + 48.44284)
catboost_koef = koef*46.75517
lgbm_koef = koef*47.82269
xgb_koef = koef*48.44284

blending_pred = xgb_pred_by_fold*xgb_koef + catboost_pred_by_fold*catboost_koef + lgbm_pred_by_fold*lgbm_koef
test['Доля сигнала в ВП'] = blending_pred
test.to_csv('public_subm.csv', index = False)

предикт на привате

In [ ]:
private_test_changed = private_test.copy()
private_test_changed.columns = columns

xgb_df = private_test_changed.drop(['num_test', 'measure_type',
                           'num_impulse', 'hor', 'ver'], axis = 1)
xgb_df = DMatrix(xgb_df)

lgbm_df = private_test_changed.drop(['measure_type',
                           'num_impulse', 'hor', 'ver'], axis = 1)

In [ ]:
lgbm_catboost_df = private_test_changed.drop(['measure_type',
                           'num_impulse', 'hor', 'ver'], axis = 1)
catboost_preds = []
for catboost_i in catboost_by_fold:
    catboost_preds.append(catboost_i.predict(lgbm_catboost_df))
catboost_pred_by_fold = np.mean(catboost_preds, axis = 0)

catboost_pred_by_fold

array([0.80228884, 0.55123995, 0.44833602, ..., 0.82106747, 0.89539021,
       0.91875393])

In [ ]:
lgbm_preds = []

for lgbm_i in lgbm_by_fold:
    lgbm_preds.append(lgbm_i.predict(lgbm_catboost_df))
lgbm_pred_by_fold = np.mean(lgbm_preds, axis = 0)
lgbm_pred_by_fold

array([0.79845362, 0.50821053, 0.46763197, ..., 0.87460383, 0.86344317,
       0.87874955])

In [ ]:
xgb_preds = []

for xgb_model in xgb_by_fold:
    xgb_preds.append(xgb_model.predict(xgb_df))
xgb_pred_by_fold = np.mean(xgb_preds, axis = 0)
xgb_pred_by_fold

array([0.71018004, 0.5776914 , 0.50806135, ..., 0.7941729 , 0.8177988 ,
       0.9679674 ], dtype=float32)

In [ ]:
blending_pred = xgb_pred_by_fold*xgb_koef + catboost_pred_by_fold*catboost_koef + lgbm_pred_by_fold*lgbm_koef


private_test['Доля сигнала в ВП'] = blending_pred
private_test.to_csv('private_subm.csv')